# Data Analysis

## READ BEFORE DEVELOPING
If you make any changes to the files in `modules` you have to take the following steps to get those changes in here:
1. Shut down `jupyter notebook`
2. Run `pip install .` on project root directory
3. Start up `jupyter notebook` and re-run all of the cells (to make sure it gets the imports and all)

In [12]:
from modules import process as prc
from pkg_resources import resource_filename

In [13]:
# Collect the raw data output from API
raw_data = prc.read_file(resource_filename('data', 'training_data.json'))

# After loading in the raw data we want to clean out any instances were the was an invalid match
data = prc.filter_bad_data(raw_data)

# Get the hero win rate dataset
hero_winrates = prc.get_hero_winrates(data, resource_filename('data', 'heroes.json'))

In [19]:
# Do calculations on data
count = 0
for result in data:
    match = result['result']
    teams = prc.determine_teams(match) #{'radiant_roster': radiant, 'dire_roster': dire}

    # calculate team networths
    radiant_networth = prc.calculate_per_min_metrics(teams['radiant_roster'], match['duration'], 'gold_per_min')
    dire_networth = prc.calculate_per_min_metrics(teams['dire_roster'], match['duration'], 'gold_per_min')
    networth_diff = radiant_networth - dire_networth

    # calculate team exp
    radiant_xp = prc.calculate_per_min_metrics(teams['radiant_roster'], match['duration'], 'xp_per_min')
    dire_xp = prc.calculate_per_min_metrics(teams['dire_roster'], match['duration'], 'xp_per_min')
    xp_diff = radiant_xp - dire_xp
    
    # calculate winrate average
    radiant_winrate = prc.calculate_winrate_average(teams['radiant_roster'], hero_winrates)
    dire_winrate = prc.calculate_winrate_average(teams['dire_roster'], hero_winrates)
    winrate_diff = radiant_winrate - dire_winrate    
    
    # get team scores
    team_scores = prc.get_team_scores(match)
    score_diff = team_scores['radiant_score'] - team_scores['dire_score']
    
    # get barracks
    radiant_barracks_alive = prc.get_number_of_standing_buildings(match['barracks_status_radiant'])
    dire_barracks_alive = prc.get_number_of_standing_buildings(match['barracks_status_dire'])
    barracks_diff = radiant_barracks_alive - dire_barracks_alive
    
    # get radiant win
    win = match['radiant_win']
    
    # put data into csv
    

    # only run for small sample size to test
    count += 1
    if count > 100:
        prc.to_csv('../data/train.csv', networth_diff, xp_diff, winrate_diff, score_diff, barracks_diff, win)
        if count == 200:
            break
        